The following exercises are meant to be solved by gathering the bash commands incrimentally in two scripts, one for ex 1.* the other for ex 2.* 

### Ex 1

1\.a Make a new directory called `students` in your home. Download a csv file with the list of students of this lab from [here](https://www.dropbox.com/s/867rtx3az6e9gm8/LCP_22-23_students.csv) (use the `wget` command) and copy that to `students`. First check whether the file is already there

1\.b Make two new files, one containing the students belonging to PoD, the other to Physics.

1\.c For each letter of the alphabet, count the number of students whose surname starts with that letter. 

1\.d Find out which is the letter with most counts.

1\.e Assume an obvious numbering of the students in the file (first line is 1, second line is 2, etc.), group students "modulo 18", i.e. 1,19,37,.. 2,20,38,.. etc. and put each group in a separate file  

```bash
# go in home and make directory
cd $HOME
mkdir students

# download file in home
if [ ! -f "./LCP_22-23_students.csv" ]
then
    wget https://www.dropbox.com/s/867rtx3az6e9gm8/LCP_22-23_students.csv
fi   

# check if file is already in students

if [ ! -f "./LCP_22-23_students.csv" ]
then
    echo "the file ./LCP_22-23_students.csv does not exist! "
else
    file=./LCP_22-23_students.csv
fi

# copy the file to students
cp LCP_22-23_students.csv students

# make two files containing different students
grep "PoD" LCP_22-23_students.csv > pod.csv
grep "Physics" LCP_22-23_students.csv > physics.csv

# counts how many students' surname begins with each letter of the alphabet
# find which letter has more counts
n_max=0
letter_max=0

for letter in {A..Z}; do 
    n=$(grep -c "^$letter" LCP_22-23_students.csv)
    echo "number of students whose surname start with letter $letter is: $n"

    if [ $n -gt $n_max ]; then
    	n_max=$n
    	letter_max=$letter
    fi
done
echo "the letter that has more counts is $letter_max"

# group students "modulo 18", i.e. 1,19,37,.. 2,20,38,.. etc. and put each group in a separate file 
#n_students=$(wc -l < LCP_22-23_students.csv)    # number of students

#awk 'NR > 1 && (NR-1) % 18 == 1 {print $0 > }' LCP_22-23_students.csv

# Create a directory to store the output files
output_dir="students"
mkdir -p "$output_dir"

# Use awk to group rows modulo 18, skipping the header and redirecting to files
awk -v output_dir="$output_dir" '
{
    if ((NR-1) % 18 == 0) {
        close(output_file)
        output_file = output_dir "/Group" ++i ".csv"
    }
    print $0 > output_file
}' LCP_22-23_students.csv

```

### Ex 2

2.a Make a copy of the file `data.csv` removing the metadata and the commas between numbers; call it `data.txt`

2\.b How many even numbers are there?

2\.c Distinguish the entries on the basis of `sqrt(X^2 + Y^2 + Z^2)` is greater or smaller than `100*sqrt(3)/2`. Count the entries of each of the two groups 

2\.d Make `n` copies of data.txt (with `n` an input parameter of the script), where the i-th copy has all the numbers divided by i (with `1<=i<=n`).

``` bash

# Make a copy of the file `data.csv` removing the metadata
# and the commas between numbers; call it `data.txt`
grep -v "^#" data.csv | tr ',' ' ' > data.txt

# Count even numbers
count=0

while IFS= read -r line; do
    for d in $line; do
    	if (( $d%2==0 )); then count=$((count+1)); fi
    done
done < data.txt

echo "The number of even numbers is $count"

# Distinguish the entries on the basis of `sqrt(X^2 + Y^2 + Z^2)` is greater or smaller than `100*sqrt(3)/2`.
# Count the entries of each of the two groups 

threshold=$(echo "100 * sqrt(3) / 2" | bc -l)
greater=0
smaller=0

while IFS= read -r line; do
    # Extracting the first three values from the line
    read -r x y z _ _ _ <<< "$line"

    # Using bc for floating-point arithmetic
    s=$(echo "sqrt($x^2 + $y^2 + $z^2)" | bc -l)

    if (( $(echo "$s > $threshold" | bc -l) )); then
        greater=$((greater + 1))
    else
        smaller=$((smaller + 1))
    fi
done < data.txt

echo "The entries with the value greater are $greater"
echo "The entries with the value smaller are $smaller"

# Make `n` copies of data.txt (with `n` an input parameter of the script),
# where the i-th copy has all the numbers divided by i (with `1<=i<=n`).
n="$1"

for i in $(seq 1 "$n"); do
	output_file="data_copy_$i.txt"
	awk -v c="$i" '{ for (j = 1; j <= NF; j++) $j = $j / c }1' data.txt > "$output_file"
done

```